In [1]:
import math
import pandas as pd
import sqlite3
import numpy as np

In [2]:
"Calculer une estimation de la population en 2050"
connexion = sqlite3.connect("factbook.db")
data = pd.read_sql_query("select * from facts where (area_land != ' ') and (area_water != 0);", con = connexion)
data.head(2)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,created_at,updated_at
0,2,al,Albania,28748.0,27398,1350,3029278.0,0.30,12.92,6.58,3.3,2015-11-01 13:19:54.431082,2015-11-01 13:19:54.431082
1,7,ar,Argentina,2780400.0,2736690,43710,43431886.0,0.93,16.64,7.33,0.0,2015-11-01 13:20:18.233063,2015-11-01 13:20:18.233063


In [3]:
data.dropna(inplace = True)

## Formule mathématiques de prévision demographique

* **N = N0*exp(rt)** >>>>>>>> N0*e**(r*t)

    * **N0 = pop initial**
    
    * **exp = fonction exponentielle**
    
    * **r = taux d'évolution annuelle**
    
    * **t = Nbre d'année entre tps prevu  et temps init**

In [4]:
def pop_growth(data):
    return data['population']*np.e**((data['population_growth']/100)*35)

In [5]:
np.e

2.718281828459045

In [6]:
data["Pop_2050"] =  data.apply(lambda row: pop_growth(row), axis = 1)

In [7]:
data.head(2)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,created_at,updated_at,Pop_2050
0,2,al,Albania,28748.0,27398,1350,3029278.0,0.30,12.92,6.58,3.3,2015-11-01 13:19:54.431082,2015-11-01 13:19:54.431082,3.364651e+06
1,7,ar,Argentina,2780400.0,2736690,43710,43431886.0,0.93,16.64,7.33,0.0,2015-11-01 13:20:18.233063,2015-11-01 13:20:18.233063,6.014112e+07


### Trier sur la zone geographique

In [8]:
data =  data[(data['area_land']> 0) &  (data['population']> 0)]
data.head(2)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,created_at,updated_at,Pop_2050
0,2,al,Albania,28748.0,27398,1350,3029278.0,0.30,12.92,6.58,3.3,2015-11-01 13:19:54.431082,2015-11-01 13:19:54.431082,3.364651e+06
1,7,ar,Argentina,2780400.0,2736690,43710,43431886.0,0.93,16.64,7.33,0.0,2015-11-01 13:20:18.233063,2015-11-01 13:20:18.233063,6.014112e+07


In [9]:
data.sort_values(by= ['Pop_2050'], ascending = False).iloc[0:9]

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,created_at,updated_at,Pop_2050
59,77,in,India,3287263.0,2973193,314070,1.251696e+09,1.22,19.55,7.32,0.04,2015-11-01 13:26:24.788193,2015-11-01 13:26:24.788193,1.918415e+09
26,37,ch,China,9596960.0,9326410,270550,1.367485e+09,0.45,12.49,7.53,0.44,2015-11-01 13:22:53.813142,2015-11-01 13:22:53.813142,1.600752e+09
96,129,ni,Nigeria,923768.0,910768,13000,1.815621e+08,2.45,37.64,12.90,0.22,2015-11-01 13:30:39.695455,2015-11-01 13:30:39.695455,4.279890e+08
137,186,us,United States,9826675.0,9161966,664709,3.213689e+08,0.78,12.49,8.15,3.86,2015-11-01 13:35:14.898271,2015-11-01 13:35:14.898271,4.222466e+08
60,78,id,Indonesia,1904569.0,1811569,93000,2.559937e+08,0.92,16.72,6.37,1.16,2015-11-01 13:26:30.435758,2015-11-01 13:26:30.435758,3.532418e+08
98,132,pk,Pakistan,796095.0,770875,25220,1.990858e+08,1.46,22.58,6.49,1.54,2015-11-01 13:30:54.514863,2015-11-01 13:30:54.514863,3.318676e+08
7,14,bg,Bangladesh,148460.0,130170,18290,1.689577e+08,1.60,21.14,5.61,0.46,2015-11-01 13:20:52.753843,2015-11-01 13:20:52.753843,2.957897e+08
15,24,br,Brazil,8515770.0,8358140,157630,2.042598e+08,0.77,14.46,6.58,0.14,2015-11-01 13:21:42.799461,2015-11-01 13:21:42.799461,2.674393e+08
28,40,cg,"Congo, Democratic Republic of the",2344858.0,2267048,77810,7.937514e+07,2.45,34.88,10.07,0.27,2015-11-01 13:23:09.778305,2015-11-01 13:23:09.778305,1.871078e+08


In [10]:
pop_total = data.population.sum()
pop_total

6867522525.0

In [19]:
ratio_area = (data['area_land'].sum())/(data['area_water'].sum())
#data['ratio_area'] = ratio_area
#data_1 = data[['name',"ratio_area"]].sort_values(by = 'ratio_area', ascending = False )

In [20]:
ratio_area

25.141513375624392

In [46]:
connexion.close()

# Requête:

Calculer le **ratio** de la surface terrestre total sur la surface totale des océans appartenant aux pays  de la table facts.
 
**Infos:**

**area_water** = surface des océans
**area_land** = surface des terres

In [13]:
"Calculer une estimation de la population en 2050"
connexion = sqlite3.connect("factbook.db")
data_2 = pd.read_sql_query("SELECT sum(area_land), sum(area_water) from facts WHERE area_land != ' ';", con = connexion)
data_2.head(2)

,sum(area_land),sum(area_water)
0,128584834,4529125


In [14]:
data = data[data['area_water']>0]

# Requête: 
Compter le nombre de valeurs non num en SQL sous python (Jupyter)

In [21]:
import sqlite3

In [22]:
connexion_sql =  sqlite3.connect("factbook.db")

In [25]:
data_sql = pd.read_sql_query("select * from facts ;", con =connexion_sql )
data_sql.head(2)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,created_at,updated_at
0,1,af,Afghanistan,652230.0,652230.0,0.0,32564342.0,2.32,38.57,13.89,1.51,2015-11-01 13:19:49.461734,2015-11-01 13:19:49.461734
1,2,al,Albania,28748.0,27398.0,1350.0,3029278.0,0.30,12.92,6.58,3.30,2015-11-01 13:19:54.431082,2015-11-01 13:19:54.431082


In [26]:
data_sql.notna().sum()

id                   261
code                 261
name                 261
area                 249
area_land            246
area_water           243
population           242
population_growth    236
birth_rate           228
death_rate           228
migration_rate       223
created_at           261
updated_at           261
dtype: int64

In [45]:
connexion_sql.close()

# Autre façon de faire

In [27]:
import sqlite3
con_sql = sqlite3.connect("factbook.db")

In [31]:
data_count_sql = con_sql.execute("select count(birth_rate) from facts ;").fetchall()
print(data_count_sql[0][0])

228


In [44]:
con_sql.close()

# Max et Min d'une colonne numerique en SQL

In [34]:
import sqlite3
con_sql_max = sqlite3.connect("factbook.db")
max_sql = con_sql_max.execute('SELECT max(birth_rate) from facts ;').fetchall()
print(max_sql)
print('-'*60)
print(max_sql[0][0])

[(45.45,)]
------------------------------------------------------------
45.45


# Exercice:

**1**. Utiliser lafonction **min** sur la colonne **population_growth**

**2**.  Utiliser lafonction **max** sur la colonne **death_rate**

In [35]:
import sqlite3
con_sql_max_death_rate = sqlite3.connect("factbook.db")
max_sqm_death = con_sql_max_death_rate.execute('SELECT max(death_rate) from facts ;').fetchall()
print(max_sqm_death)
print('-'*60)
print(max_sqm_death[0][0])

[(14.89,)]
------------------------------------------------------------
14.89


In [40]:
con_sql_max_death_rate.close()

In [36]:
import sqlite3
con_sql_min_pop = sqlite3.connect("factbook.db")
min_sqm_pop = con_sql_min_pop.execute('SELECT min(population_growth) from facts ;').fetchall()
print(min_sqm_pop)
print('-'*60)
print(min_sqm_pop[0][0])

[(0.0,)]
------------------------------------------------------------
0.0


In [41]:
con_sql_min_pop.close()

# Exercice:

**1**. Utiliser la fonction **sum** pour trouver le total de la colonne **area_land**

**2**.  Utiliser la fonction **avg** pour trouver la **moyenne** de la colonne **area_water**

In [37]:
import sqlite3
con_sql_sum_area = sqlite3.connect("factbook.db")
sum_sql_area = con_sql_saum_area.execute('SELECT sum(area_land) from facts ;').fetchall()
print(sum_sql_area)
print('-'*60)
print(sum_sql_area[0][0])

[(128584834,)]
------------------------------------------------------------
128584834


In [39]:
import sqlite3
con_sql_moy_area_water = sqlite3.connect("factbook.db")
moy_sql_area_water = con_sql_moy_area_water.execute('SELECT avg(area_water) from facts ;').fetchall()
print(moy_sql_area_water)
print('-'*60)
print(moy_sql_area_water[0][0])

[(19067.59259259259,)]
------------------------------------------------------------
19067.59259259259


In [42]:
con_sql_sum_area.close()

In [43]:
con_sql_moy_area_water.close()

# Exercice: 

Ecrire une reqête qui calcule:

- La **moyenne** de la colonne **population**

- La **somme** de la colonne **popution**

- Le **maximum** de la colonne **birth_rate**

A faire sur DBBROWSER et Jypiter Notebook

Afficher les 3 résultats que vous assignerez à trois variables différentes.

In [86]:
import sqlite3
connexion = sqlite3.connect("factbook.db")
data_fct = pd.read_sql_query("SELECT avg(population), sum(population), max(birth_rate) from facts;", con =connexion )

In [53]:
data_fct.head()

,avg(population),sum(population),max(birth_rate)
0,6.209493e+07,15026972654,45.45


# Autre facon

In [74]:
import sqlite3
connexion = sqlite3.connect("factbook.db")
data_fct = connexion.execute("SELECT avg(population), sum(population), max(birth_rate) from facts;").fetchall()

In [75]:
moy = data_fct[0][0]
sum_ = data_fct[0][1]
max_ = data_fct[0][2]

In [80]:
print("La moyenne: {}, \nLa somme est: {}, \nLe maximum est: {}." .format(moy, sum_, max_))

La moyenne: 62094928.32231405, 
La somme est: 15026972654, 
Le maximum est: 45.45.


# Exercice: 

Ecrire une requête qui calcule:

La **moyenne** de la colonne **population_growth** pour les oays ayant une pop sup à **10 000 000**

A faire sur DBBROWSER et Jypiter Notebook

In [84]:
import sqlite3
connexion =  sqlite3.connect("factbook.db")
data_2 =  connexion.execute("SELECT avg(population_growth) from facts where  population > 10000000 ;").fetchall()
print(data_2[0][0])

1.4572222222222226


In [85]:
connexion =  sqlite3.connect("factbook.db")
data_3 =  connexion.execute("SELECT DISTINCT birth_rate from facts;").fetchall()
print(data_3)

[(38.57,), (12.92,), (23.67,), (8.13,), (38.78,), (15.85,), (16.64,), (13.61,), (12.15,), (9.41,), (15.5,), (13.66,), (21.14,), (11.87,), (10.7,), (11.41,), (24.68,), (36.02,), (17.78,), (22.76,), (8.87,), (20.96,), (14.46,), (17.32,), (8.92,), (42.03,), (18.39,), (42.01,), (23.83,), (36.17,), (10.28,), (20.33,), (35.08,), (36.6,), (13.83,), (12.49,), (16.47,), (27.84,), (34.88,), (35.85,), (15.91,), (28.67,), (9.45,), (9.9,), (9.63,), (10.27,), (23.65,), (15.41,), (18.73,), (18.51,), (22.9,), (16.46,), (33.31,), (30.0,), (10.51,), (37.27,), (19.43,), (10.72,), (12.38,), (34.49,), (30.86,), (12.74,), (8.47,), (31.09,), (8.66,), (16.03,), (24.89,), (35.74,), (33.38,), (15.59,), (22.31,), (23.14,), (9.16,), (13.91,), (19.55,), (16.72,), (17.99,), (31.45,), (14.84,), (18.48,), (8.74,), (18.16,), (7.93,), (25.37,), (19.15,), (26.4,), (21.46,), (14.52,), (8.19,), (None,), (19.91,), (22.98,), (24.25,), (10.0,), (14.59,), (25.47,), (34.41,), (18.03,), (10.45,), (10.1,), (11.37,), (11.55,), (3

# Exercice: 

Ecrire une requête qui calcule:

-  **moyenne** des valeurs distinctes de la colonne **birth_rate** pour laquelle **population** est plus grande que **20 millions**

- La **somme** de toutes les valeus distinctes de la colonne **popution** pour laquelle **area_land** est plus grande que **1.000.000**

A faire sur DBBROWSER et Jypiter Notebook

Afficher les 3 résultats que vous assignerez à trois variables différentes.

In [88]:
import sqlite3
con = sqlite3.connect("factbook.db")
data_4 = con.execute("SELECT avg(distinct birth_rate) from facts WHERE (population > 20000000);").fetchall()
print(data_4[0][0])

20.43473684210527


In [89]:
import sqlite3
con = sqlite3.connect("factbook.db")
data_5 = con.execute("SELECT  sum(distinct population) from facts WHERE (area_land > 1000000);").fetchall()
print(data_5[0][0])

4233873015


# Exercice :

Afficher les valeurs de la **population_growth** en millions (dec)

sur DBBROWSER et Jypiter

In [91]:
import sqlite3
con = sqlite3.connect("factbook.db")
data_6 = con.execute("SELECT (population_growth/1000000) from facts;").fetchall()
print(data_6)

[(2.32e-06,), (3e-07,), (1.8400000000000002e-06,), (1.2e-07,), (2.7799999999999996e-06,), (1.24e-06,), (9.300000000000001e-07,), (1.5e-07,), (1.0700000000000001e-06,), (5.5e-07,), (9.6e-07,), (8.5e-07,), (2.4100000000000002e-06,), (1.6000000000000001e-06,), (3.1e-07,), (2.0000000000000002e-07,), (7.6e-07,), (1.87e-06,), (2.7799999999999996e-06,), (1.1100000000000002e-06,), (1.56e-06,), (1.3e-07,), (1.21e-06,), (7.7e-07,), (1.6200000000000002e-06,), (5.8e-07,), (3.03e-06,), (1.01e-06,), (3.28e-06,), (1.5800000000000001e-06,), (2.5899999999999998e-06,), (7.5e-07,), (1.3600000000000001e-06,), (2.13e-06,), (1.89e-06,), (8.2e-07,), (4.5000000000000003e-07,), (1.04e-06,), (1.77e-06,), (2.4500000000000003e-06,), (2e-06,), (1.22e-06,), (1.91e-06,), (1.3e-07,), (1.5e-07,), (1.4299999999999999e-06,), (1.6e-07,), (2.2e-07,), (2.2e-06,), (2.1e-07,), (1.23e-06,), (1.35e-06,), (1.79e-06,), (2.5e-07,), (2.5099999999999997e-06,), (2.25e-06,), (5.5e-07,), (2.8900000000000003e-06,), (6.7e-07,), (4.00000

# Requête: 

Ecrire une requête qui calcule, pour chaque pays:

-  **Le nombre dhabitant** qu'il aura l'année suivante

A faire sur DBBROWSER et Jypiter Notebook

**NB**: Multiplier les colonne pop et population_growth + population

### 1ière methode:

In [93]:
import sqlite3
con = sqlite3.connect("factbook.db")
data_7 = con.execute("select (population*population_growth) + population from facts;").fetchall()
print(data_7)

[(108113615.44,), (3938061.4,), (112299751.44,), (95849.6,), (74183834.34,), (207056.64,), (83823539.98,), (3514839.3,), (47094598.980000004,), (13431602.5,), (19170328.799999997,), (600504.45,), (4591950.33,), (439290137.0,), (380691.24,), (11507626.8,), (19930192.48,), (996949.03,), (39495885.66,), (1565449.09,), (27650257.92,), (4369772.15,), (4823808.99,), (361539867.24,), (1125672.52,), (11355290.94,), (76294694.58,), (113203614.06,), (45976941.28,), (40528590.480000004,), (85223792.62,), (61424713.0,), (1288543.48,), (16875517.07,), (33614907.84,), (31865033.2,), (1982853812.6,), (95342925.12,), (2163289.67,), (273844219.20000005,), (14265291.0,), (10687399.68,), (67789328.82,), (5045273.72,), (12686147.95,), (2889748.71,), (12348016.72,), (6809433.66,), (2650636.8,), (89064.47,), (23367625.88,), (37290730.6,), (246879834.84,), (7676687.5,), (2600007.9299999997,), (21214989.25,), (1961401.0,), (386922035.91,), (1518679.63,), (7667690.800000001,), (95171885.38,), (4996634.48,), (6

# Deuxième methode

In [98]:
con = sqlite3.connect('factbook.db')
data_8 =pd.read_sql_query( "select name , (population*population_growth) + population from facts;",con = con )

In [100]:
data_8.head(9)

,name,(population*population_growth) + population
0,Afghanistan,1.081136e+08
1,Albania,3.938061e+06
2,Algeria,1.122998e+08
3,Andorra,9.584960e+04
4,Angola,7.418383e+07
5,Antigua and Barbuda,2.070566e+05
6,Argentina,8.382354e+07
7,Armenia,3.514839e+06
8,Australia,4.709460e+07


## Requête:

### Calcul de moyennne: Calcul de moyenne sur la table fact de la base de données factbook.db
- En SQL, calculer les **moyennes** des colonnes, **Population, Population_growth, Birth_rate et death_rate** 
puis en python executer cette requête SQL

- Recupérer les résultats dans les variables: pop_avg, pop_growth_avg, birth_rate_avg et death_rate_avg  et les afficher

In [11]:
import sqlite3
import pandas as pd
connexion =  sqlite3.connect("factbook.db")
data_8 = pd.read_sql_query('SELECT round(avg(Population), 2) as pop_avg, round(avg(Population_growth),2) as pop_growth_avg,round(avg(birth_rate), 2) as birth_rate_avg, round(avg(death_rate), 2) as death_rate_avg from facts;',
                          con  = connexion)

In [12]:
data_8.head()

,pop_avg,pop_growth_avg,birth_rate_avg,death_rate_avg
0,62094928.32,1.2,19.33,7.82


In [13]:
import sqlite3
connexion =  sqlite3.connect("factbook.db")
data_9 = connexion.execute('SELECT round(avg(Population), 2) as pop_avg, round(avg(Population_growth),2) as pop_growth_avg,round(avg(birth_rate), 2) as birth_rate_avg, round(avg(death_rate), 2) as death_rate_avg from facts;').fetchall()

In [14]:
print(data_9)

[(62094928.32, 1.2, 19.33, 7.82)]


In [18]:
pop_avg = data_9[0][0]
pop_growth_avg = data_9[0][1]
birth_rate_avg = data_9[0][2]
death_rate_avg = data_9[0][3]

print("La population moyenne est : {}, \nLa population_growth moyenne est: {}, \nLa birth_rate moyenne est: {} et,\nLa death_rate moyenne est: {} " .format(pop_avg,pop_growth_avg,birth_rate_avg, death_rate_avg))

La population moyenne est : 62094928.32, 
La population_growth moyenne est: 1.2, 
La birth_rate moyenne est: 19.33 et,
La death_rate moyenne est: 7.82 


## Exercice 52: Calcul de Max et min :
#### Calcul de de caleurs min et max sur la table fact de la base de données factbook.db
* En SQL, calculer les les valeus minimums et maximums  des colonnes, Population, Population_growth, Birth_rate et death_rate (2 requête SQL bien choisie).

* Recupérer les résultats dans les variables: pop_min,pop_max, pop_growth_min, pop_growth_max,  birth_rate_min, birth_rate_max, death_rate_min et death_rate_max  et les afficher



In [30]:
 min_max_sql = 'SELECT min(Population), max(Population) , min(Population_growth), max(Population_growth), min(birth_rate), max(birth_rate) ,min(death_rate), max(death_rate) from facts;'

In [31]:
import sqlite3
connexion =  sqlite3.connect("factbook.db")
data_10 = connexion.execute(min_max_sql).fetchall()

In [32]:
print(data_10)

[(0, 7256490011, 0.0, 4.02, 6.65, 45.45, 1.53, 14.89)]


In [33]:
pop_min = data_10[0][0]
pop_max = data_10[0][1]
pop_growth_min = data_10[0][2]
pop_growth_max = data_10[0][3]
birth_rate_min = data_10[0][4]
birth_rate_max = data_10[0][5]
death_rate_min = data_10[0][6]
death_rate_max = data_10[0][7]
print(pop_min)
print(pop_max)
print(pop_growth_min)
print(pop_growth_max)
print(birth_rate_min)
print(birth_rate_max)
print(death_rate_min)
print(death_rate_max)

0
7256490011
0.0
4.02
6.65
45.45
1.53
14.89


## Exercice : Prédire la future croissance démographique

* Retourner les valeurs de la croissance estimées l'année suivante en prenant en compte les remarques suivantes:

    - Arrondir les valeurs à l'entier près (Valeurs non décimales)
    - Filtrer les lignes qui contiennent des valeurs NULL, comme valeurs pour les colonnes population ou 
    - population_growth (population is not null)
    - restrindre la requête aux pays ayant une population inférieur à 7 milliards,  et supérieure à 0
    
* Asigner le resultats à la variable projected_population

* Afficher les 10 premieres valeurs du résultats

### Rappel : Projected_population =  population*(1 + population_growth/100)

In [43]:
pop_proj = "SELECT  round(population*(1 + population_growth/100), 0)  from facts where (Population  < 7000000000) and (population > 0)  and population is not null and population_growth is not null  LIMIT 10 ;"
import sqlite3
connexion =  sqlite3.connect("factbook.db")
data_11 = connexion.execute(pop_proj).fetchall()

In [44]:
print(data_11)

[(33319835.0,), (3038366.0,), (40269742.0,), (85683.0,), (20170938.0,), (93582.0,), (43835803.0,), (3060967.0,), (22994450.0,), (8713211.0,)]


## Requête : Explorer pus d'estimation demographique

Dernière mission du challenge 2: 

* Ecrire une requête qui retourne:

    * La valeur **min** de l'estimation de la population etl'assigner à la variable proj_pop_min
    * La valeur **max** de l'estimation de la population etl'assigner à la variable proj_pop_max
    * La valeur **moyenne** de l'estimation de la population etl'assigner à la variable proj_pop_avg
    
* Afficher les résultats


In [48]:
import sqlite3
con_demo =  sqlite3.connect("factbook.db")
min_max_avg_pop_proj =  "SELECT  round(min(population*(1 + population_growth/100)), 2),  round(max(population*(1 + population_growth/100)),2), round(avg(population*(1 + population_growth/100)), 2) from facts where (Population  < 7000000000) and (population > 0) and population_growth is not null and population is not null;"
data_11 = con_demo.execute(min_max_avg_pop_proj).fetchall()

In [50]:
print("La min:", data_11[0][0])
print("La max:", data_11[0][1])
print("La moyenne:", data_11[0][2])

La min: 48.0
La max: 1373639072.25
La moyenne: 33405469.3
